# 9. Summary and QR Decomposition

### me

- https://sites.google.com/site/mathxuemei/research

###  Rest of the course

- Github Repo: https://github.com/Numerical-Analysis/MSAN18SUM_NLA

## Important Linear Algebra Problems

### 1 Solve $Ax=b$

- LU (Gauss Elilmination): direct method. $O(n^3)$
  
  Numerically unstable without pivoting. 
  
  see Section 4.2 of [my notes](https://github.com/Numerical-Analysis/course/blob/master/NumericalMethodsNotes.pdf)
  
  Cholesky factorization is a special case of LU: $A$ is positive definite. STABLE.
  
- May resort to iterative methods when system is too large.


### 2 The Least Squares problem $\min_x \|Ax-b\|_2^2$

#### 2.a Solve the normal equation $A^TAx=A^Tb$ via Cholesky factorization

In [133]:
import numpy as np
import scipy as sp
#from scipy.linalg import cholesky, solve_triangular

   
def ls_chol(A,b):
    L = sp.linalg.cholesky(A.T@A, lower=True)  #LL'x = A'b
    y = sp.linalg.solve_triangular(L, A.T@b, lower=True) #L'x=y
    return sp.linalg.solve_triangular(L.T, y)

In [142]:
A1 = np.array([[1,2],[2,3],[-1,2]])
b1 = np.array([1,2,-2])
np.allclose(ls_chol(A1,b1),np.linalg.solve(A1.T@A1,A1.T@b1))

True

#### 2.b Use QR factorization

- compute the reduced QR: $A = QR$ 
- compute $Q^Tb$
- solve $Rx = Q^Tb$
  
  Justification:
  
  $Ax=Pb$
  
  $QRx=QQ^Tb$

In [135]:
def ls_qr(A,b):
    Q,R = sp.linalg.qr(A, mode='economic')
    return sp.linalg.solve_triangular(R, Q.T@b)

In [140]:
print(np.linalg.solve(A1.T@A1,A1.T@b1))

[ 1.43939394 -0.27272727]


#### 2.c Use SVD factorization

- compute the reduced SVD: $A = U\Sigma V^T$
- compute $U^Tb$
- solve $\Sigma y = U^Tb$
- $x = Vy$
 
 Justification?
 
 $Ax=Pb$
 
 $P=UU^T$

In [150]:
def ls_svd(A,b):
    U,S,Vt = sp.linalg.svd(A, full_matrices = False)  #I hate that python svd spits out V^T
    m,n = A.shape
    #y = (U.T@b)/(S.reshape((n,1)))
    y = U.T@b/S
    
    return Vt.T@y

In [151]:
print(ls_svd(A1,b1))

[ 1.43939394 -0.27272727]


#### Numerical Comparison

In [117]:
# setting up A and b from Trefethen 
import numpy as np
m = 60
n = 10
t = np.arange(0,1+1.0/(m-1),1.0/(m-1)) # Set t to a discretization of [0,1]. len(t)=m
A = np.array([t**i for i in range(n)]).T # Construct a submatrix of a Vandermonde matrix
#size of A is m by n

#truex is the real least square solution
truex = np.array([  1.1454379,
   0.52179532, -2.59420408,  0.0355606,   1.67058624,  0.1212572,  -1.14884385,
  -0.78537181, -1.18751783, 1])

#Construct the right-hand side vector b so that least square solution of Ax=b is truex
U,S,V = np.linalg.svd(A)
AC = U[:,n:]
b = A.dot(truex).reshape(m,1) + AC@np.ones([m-n,1])

In [141]:
np.linalg.cond(A)

3599969.1117646005

In [126]:
import timeit
import pandas as pd
row_names = ['Normal Eqns- Cholesky', 
             'QR Factorization', 
             'SVD',
             'Scipy lstsq']
             

name2func =  {'Normal Eqns- Cholesky': 'ls_chol', 
             'QR Factorization': 'ls_qr',
             'SVD': 'ls_svd',
             'Scipy lstsq': 'scipylstq'}
pd.options.display.float_format = '{:,.6f}'.format
df = pd.DataFrame(index=row_names)
#df_error = pd.DataFrame(index=row_names, columns=index)
def scipylstq(A, b):
    return scipy.linalg.lstsq(A,b)[0]

In [132]:
for name in row_names:
    fcn = name2func[name]
    t = timeit.timeit(fcn + '(A,b)', number=50, globals=globals())
    coeffs = locals()[fcn](A, b)
    df.set_value(name, 'Time (s)',t)
    df.set_value(name, 'Error', np.abs(1 - coeffs[-1]))
df             

,Time (s),Error
Normal Eqns- Cholesky,0.011607,0.000026
QR Factorization,0.010365,0.000000
SVD,0.012659,0.000000
Scipy lstsq,0.027934,0.000000


#### Summary of the 3 least sqares algorithms

Cholesky is usually fastest, but least stable

SVD is usually the most stable, but takes longer

QR is in the middle: recommended for daily use

#### Remarks on stability

- The stability of the least square problem (has nothing to do with computer) depends on the condition number of $A$.

- Any intuition why Least squares via SVD or QR is more stable?

### 3 Eigenvalue problem: problem itself is not stable

- No direct method. 
- Can only use iterative methods: only approximating.
 
  ```diff
  -"Direct methods can solve a problem exactly in a finite number of steps if implemented in exact arithmetic. By contrast, any eigenvalue solver must be iterative." - Trefethen and Bau
  ```
- Stable if matrix is symmetric.
- **use QR factorization**

### 4 Finding SVD

- Randomized SVD uses QR factorization - Rachel's notebook 4

## How to do QR decomposition?

### (Block matrix multiplication again)

Useful for matrix manipulation: svd/qr, left/right multiple of diagonal matrix, write matrix as sum of rank-1,...

Has memory advantage: Rachel's notebook 5

See Section 3.2 of my notes

### Gram-Schmidt algorithm is a way to find (reduced) QR, but not stable

See Example 6.6 of my notes if you want to know more details.

### Householder Reflector is the better way

Section 6.3

Given a vector $u$, $H_u(x)$ is the reflection of $x$ with respect to $u^\perp$. $H_u$ is the operator, and

$H_u = I - 2\frac{uu^T}{u^Tu}$

And $H_u$ is orthogonal, and symmetric

\begin{align*}\left[\begin{array}{cccc}
\times&\times&\times\\
\times&\times&\times\\
\times&\times&\times\\
\times&\times&\times\\
\times&\times&\times
\end{array}\right]\xrightarrow{Q_1}\left[\begin{array}{cccc}
\times&\times&\times\\
0&\times&\times\\
0&\times&\times\\
0&\times&\times\\
0&\times&\times
\end{array}\right]\xrightarrow{Q_2}\left[\begin{array}{cccc}
\times&\times&\times\\
&\times&\times\\
&0&\times\\
&0&\times\\
&0&\times
\end{array}\right]\xrightarrow{Q_3}\left[\begin{array}{cccc}
\times&\times&\times\\
&\times&\times\\
&&\times\\
&&0\\
&&0
\end{array}\right]\end{align*}

$\hspace{7.3cm}A_0:=A\hspace{1.8cm}A_1:=Q_1A\hspace{1.2cm}A_2:=Q_2Q_1A\hspace{0.8cm}A_3:=Q_3Q_2Q_1A$

HW3: How to find $Q_2$?

We will see Householder reflector again in eigenvalue problems.

$R = Q_3Q_2Q_1A$

$Q_3^TR = Q_3^TQ_3Q_2Q_1A$, ...

$Q_1^TQ_2^TQ_3^TR=A$